In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape}')
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [4]:
# Get cpu or gpu device for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using: {device} device')

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using: cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'Loss: {loss:7f}  [{current:5d} / {size:5d}]')

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f'Test error:\nAccuracy: {(100 * correct):.1f}%, Avg loss: {test_loss:.8f}')

In [8]:
epochs = 5

for t in range(epochs):
    print(f'Epoch: {t + 1}\n----------------------------------------')

    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print('Done')

Epoch: 1
----------------------------------------
Loss: 2.302387  [    0 / 60000]
Loss: 2.297080  [ 6400 / 60000]
Loss: 2.272887  [12800 / 60000]
Loss: 2.274221  [19200 / 60000]
Loss: 2.251128  [25600 / 60000]
Loss: 2.220357  [32000 / 60000]
Loss: 2.233600  [38400 / 60000]
Loss: 2.197435  [44800 / 60000]
Loss: 2.197772  [51200 / 60000]
Loss: 2.166790  [57600 / 60000]
Test error:
Accuracy: 35.5%, Avg loss: 2.15488840
Epoch: 2
----------------------------------------
Loss: 2.165476  [    0 / 60000]
Loss: 2.149814  [ 6400 / 60000]
Loss: 2.091808  [12800 / 60000]
Loss: 2.118679  [19200 / 60000]
Loss: 2.052146  [25600 / 60000]
Loss: 1.999646  [32000 / 60000]
Loss: 2.029560  [38400 / 60000]
Loss: 1.952395  [44800 / 60000]
Loss: 1.970995  [51200 / 60000]
Loss: 1.896081  [57600 / 60000]
Test error:
Accuracy: 59.3%, Avg loss: 1.88350175
Epoch: 3
----------------------------------------
Loss: 1.922217  [    0 / 60000]
Loss: 1.874384  [ 6400 / 60000]
Loss: 1.766987  [12800 / 60000]
Loss: 1.815954

In [9]:
torch.save(model.state_dict(), 'models/model.pth')
print('Saved pytorch model state to models/model.pth')

Saved pytorch model state to models/model.pth


In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load('models/model.pth'))

<All keys matched successfully>

In [11]:
classes = [
    'T-shirt/top',
    'Trouse',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

model.eval()

x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = model(x)
    
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
